importing libraries

In [1]:
import pandas as pd
import nltk
from nltk.corpus import twitter_samples
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier

dataset reading

In [2]:
dataset1 = pd.read_csv('Twitter_Data.csv')
dataset2 = pd.read_csv('apps.csv')
dataset1.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


Load positive and negative tweets from the NLTK Twitter dataset

In [3]:
positive_tweets = dataset1[dataset1['category'] == 'positive']['clean_text'].tolist()
negative_tweets = dataset1[dataset1['category'] == 'negative']['clean_text'].tolist()

Preprocess the data

In [4]:
def preprocess_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    
    # Tokenize, remove stopwords, and perform stemming
    tokens = word_tokenize(tweet)
    tokens = [ps.stem(word.lower()) for word in tokens if word.isalpha() and word.lower() not in stop_words]
    
    return ' '.join(tokens)

In [5]:
positive_tweets = [(preprocess_tweets(tweet), 'positive') for tweet in positive_tweets]
negative_tweets = [(preprocess_tweets(tweet), 'negative') for tweet in negative_tweets]

Sentiment Analysis Model Development and Feature Engineering

In [6]:
# Split the data into training and testing sets
training_data = positive_tweets[:4000] + negative_tweets[:4000]
testing_data = positive_tweets[4000:] + negative_tweets[4000:]

# Create feature sets
all_words = [word for tweet, sentiment in training_data for word in tweet.split()]
word_freq = FreqDist(all_words)
common_words = word_freq.most_common(3000)
features = [word for word, freq in common_words]

def extract_features(tweet):
    words = set(tweet)
    features_dict = {}
    for word in features:
        features_dict[word] = (word in words)
    return features_dict



In [7]:
from nltk.probability import LaplaceProbDist

In [8]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [9]:
for word in features:
    if word not in word_freq:
        word_freq[word] = 0

training_features = [(extract_features(tweet), sentiment) for tweet, sentiment in training_data]
testing_features = [(extract_features(tweet), sentiment) for tweet, sentiment in testing_data]

# Train the Naive Bayes classifier with Laplace smoothing
from nltk.classify.util import apply_features
training_features = apply_features(extract_features, training_data)

classifier = NaiveBayesClassifier.train(training_features)

# Evaluate the classifier
accuracy = nltk.classify.accuracy(classifier, testing_features)
print(f'Accuracy: {accuracy}')

ValueError: A ELE probability distribution must have at least one bin.